In [31]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
  .remote("sc://192.168.1.7:15002") \
  .appName("UDFTransformation") \
  .config("spark.sql.ansi.enabled", "false") \
  .config("spark.sql.execution.pythonUDF.arrow.enabled", "true") \
  .config("spark.sql.repl.eagerEval.enabled", "true") \
  .config("spark.sql.repl.eagerEval.truncate", "100") \
  .getOrCreate()

spark

In [ ]:
from pyspark.sql.utils import IllegalArgumentException

# lines = spark \
#     .readStream \
#     .format("rate") \
#     .option("host", "192.168.1.7") \
#     .option("port", 9999) \
#     .load()

lines = (spark.readStream
  .format("rate")
  .option("rowsPerSecond", 3)
  .load()
)
try:
  query = (lines.writeStream
    .outputMode("update")
    .format("memory")
    .queryName("mem_results")
    .start()
  )
except IllegalArgumentException:
  pass # already started
import time

if True:
  for _ in range(20):
    print(f"{str(query.status):<100}", end="\r")
    time.sleep(0.1) 

In [35]:
spark.sql("select * from mem_results order by value desc")

timestamp,value
2025-10-25 19:19:26.691,23
2025-10-25 19:19:26.357,22
2025-10-25 19:19:26.024,21
2025-10-25 19:19:25.691,20
2025-10-25 19:19:25.357,19
2025-10-25 19:19:25.024,18
2025-10-25 19:19:24.691,17
2025-10-25 19:19:24.357,16
2025-10-25 19:19:24.024,15
2025-10-25 19:19:23.691,14


In [34]:
query.awaitTermination(timeout=10)

KeyboardInterrupt: 

In [ ]:
query.stop()